In [ ]:
import os
os.environ["PYTORCH_JIT_USE_NNC_NOT_NVFUSER"] = "1"
from typing import List, Tuple, Optional, Union, Iterable, NamedTuple, Any, Sequence

from tqdm import tqdm
import yaml

import torch
from torchvision.transforms import Compose
from torch.utils.data import DataLoader

from e3nn import o3

from diffusion_edf.data import DemoSeqDataset, DemoSequence, TargetPoseDemo, PointCloud
from diffusion_edf.gnn_data import FeaturedPoints, merge_featured_points, GraphEdge
from diffusion_edf import train_utils
from diffusion_edf import preprocess
from diffusion_edf.feature_extractor import UnetFeatureExtractor
from diffusion_edf.tensor_field import TensorField

In [ ]:
device = 'cuda:0'
task = 'pick'
eval = True
compile = False

model_configs_dir = 'configs/test/model_configs.yaml'
train_configs_dir = 'configs/test/train_configs.yaml'

# Load configs, preprocessor, and dataloader

In [ ]:
with open(model_configs_dir) as file:
    model_configs = yaml.load(file, Loader=yaml.FullLoader)
with open(train_configs_dir) as file:
    train_configs = yaml.load(file, Loader=yaml.FullLoader)

In [ ]:
proc_fn = []
for proc in train_configs['preprocess_config']:
    proc_fn.append(
        getattr(preprocess, proc['name'])(**proc['configs'])
    )
proc_fn = Compose(proc_fn)

In [ ]:
collate_fn = train_utils.get_collate_fn(task=task, proc_fn=proc_fn)
trainset = DemoSeqDataset(dataset_dir=train_configs['dataset_dir'], annotation_file=train_configs['annotation_file'], device=device)
train_dataloader = DataLoader(trainset, shuffle=True, collate_fn=collate_fn, batch_size=train_configs['n_batches'])

# Load Model

In [ ]:
fe_kwargs = model_configs['scene']['feature_extractor_configs']
tf_kwargs = model_configs['scene']['tensor_field_configs']

feature_extractor = UnetFeatureExtractor(**fe_kwargs).to(device)
tf_kwargs['irreps_input'] = str(feature_extractor.irreps_output)
tf = tensor_field = TensorField(**tf_kwargs).to(device)
if compile:
    feature_extractor = torch.jit.script(feature_extractor)
    tf = torch.jit.script(tf)
if eval:
    feature_extractor = feature_extractor.eval()
    tf = tf.eval()

# Loop Example

In [ ]:
for demo_batch in train_dataloader:
    B = len(demo_batch)
    scene_pcd, grasp_pcd, target_poses = train_utils.flatten_batch(demo_batch=demo_batch) # target_poses: (Nbatch, Ngrasps, 7)
    input_pcd = FeaturedPoints(
        x=scene_pcd.x/model_configs['unit_len'],
        f=scene_pcd.f,
        b=scene_pcd.b
    )
    break

In [ ]:
output = feature_extractor(input_pcd)

In [ ]:
Nq = 20
query_x = output[0].x.detach().mean(dim=0).expand(Nq,-1)
query_x = query_x + (0.1 * torch.randn_like(query_x) * output[0].x.detach().std(dim=0))
query_points = FeaturedPoints(x=query_x, f=torch.empty_like(query_x), b=torch.zeros(len(query_x), dtype=torch.long, device=device))

In [ ]:
field_output = tf(query_points,
                  input_points = output[-1],
                  time_emb = torch.randn(B,tf.time_emb_dim, device=device))

In [ ]:
field_output

In [ ]:
sdfa

# Rotate

In [ ]:
# scale = 3
# node_coord = output[scale].x
# node_feature = output[scale].f
# node_batch = output[scale].b

In [ ]:
from diffusion_edf.transforms import quaternion_apply, random_quaternions
from diffusion_edf.gnn_data import TransformPcd

transform_input = torch.jit.script(TransformPcd(irreps=model_configs['scene']['feature_extractor_configs']['irreps_input'], device=device))
transform_output = torch.jit.script(TransformPcd(irreps=model_configs['scene']['feature_extractor_configs']['irreps_output'], device=device))

rot = random_quaternions(1, device=device)
trans = torch.randn(1,3,device=device)
Ts = torch.cat([rot,trans], dim=-1)

In [ ]:
T_idx = 0
scale_idx = 3

input_pcd_rot: FeaturedPoints = transform_input(input_pcd, Ts)
input_pcd_rot = FeaturedPoints(x=input_pcd_rot.x[T_idx], f=input_pcd_rot.f[T_idx], b=input_pcd_rot.b[T_idx])

pre_rot_out = feature_extractor(input_pcd_rot)[scale_idx]
post_rot_out: FeaturedPoints = transform_output(output[scale_idx], Ts)
post_rot_out = FeaturedPoints(x=post_rot_out.x[T_idx], f=post_rot_out.f[T_idx], b=post_rot_out.b[T_idx], w=post_rot_out.w[T_idx])

In [ ]:
isclose = torch.isclose(pre_rot_out.x, post_rot_out.x, atol=0.01, rtol=0.01)
print(f"Equivariance ratio: {(isclose.sum() / len(pre_rot_out.x.view(-1))).item()}") # Slight non-equivariance comes from FPS downsampling algorithm.

In [ ]:
isclose = torch.isclose(pre_rot_out.f, post_rot_out.f, atol=0.001, rtol=0.001)
print(f"Equivariance ratio: {(isclose.sum() / len(pre_rot_out.f.view(-1))).item()}") # Slight non-equivariance comes from FPS downsampling algorithm.